# CLIP Search over video

#### Enter Youtube link & add it to Index
#### Enter Query -> Get Frame Timestamps & Video Name back

#### Made by Ayush Yadav -> ayushyadavcodes@gmail.com

In [ ]:
pinecone_api_key = input('Enter pinecone api key')

#Setup Project
if you are using your own Pinecone account, create a new index called 'frames', with 'dot product similarity' & '512 dimensions'

else ask for my api key

In [1]:
#install all dependencies
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers
!pip install pinecone-client
!pip install pytube

Looking in indexes: https://download.pytorch.org/whl/cu118, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 31.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 10.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 7.9 MB/s eta 0:00:00


In [2]:
#connect to google drive => to works as storage
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#setup pinecon index => Vector Database
import cv2
import numpy as np
import pinecone
pinecone.init(pinecone_api_key,environment="us-central1-gcp")
index = pinecone.Index("frames")
index.describe_index_stats()

/usr/local/lib/python3.9/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


{'dimension': 512,
 'index_fullness': 0.0,
 'namespaces': {'sample': {'vector_count': 5}},
 'total_vector_count': 5}

In [4]:
#Import CLIP & Preprocessors for Image & Text

from transformers import CLIPTokenizerFast, CLIPProcessor, CLIPModel
import torch

# if you have CUDA or MPS, set it to the active device like this
device = "cuda" if torch. cuda.is_available() else \
("mps" if torch. backends.mps.is_available() else "cpu")

model_id = "openai/clip-vit-base-patch32" 

model = CLIPModel.from_pretrained(model_id).to(device)

tokenizer = CLIPTokenizerFast.from_pretrained(model_id)

processor = CLIPProcessor.from_pretrained(model_id)


# Enter Namespace Name for saving videos & Yt link
Note: remember the namespace if you want to delete it later

In [22]:
namespace_name = input("Enter Namespace name to save embeddings to")
# Get the YouTube link from user input
youtube_link = input("Enter the YouTube link: ")

Enter Namespace name to save embeddings toanimals
Enter the YouTube link: https://youtu.be/Ma5v_XTKjEA


# Download Video, Convert to Frames, convert to embedding, add to Piencone

In [23]:
#Download youtube Video 
import os
import pytube



# Create the output directory if it doesn't exist
output_location = "/content/drive/MyDrive/clippy/"
os.makedirs(output_location, exist_ok=True)

# Download the video
youtube = pytube.YouTube(youtube_link)
video_streams = youtube.streams.filter(res="360p")
video = video_streams[0]
video_size = video.filesize
video_title = video.title
print(f"Downloading {video.title} ({video_size/1e6:.2f} MB)")
video.download(output_path=output_location, filename=video_title + ".mp4")
print("Download complete.")

Download complete.


In [24]:
# Extract frames from The Downloaded Video


import os
import cv2
from tqdm.auto import tqdm

video_location = f"/content/drive/MyDrive/clippy/{video_title}.mp4"
output_location = "/content/drive/MyDrive/clippy/output/{title}".format(title=video_title)

# Create the output directory if it doesn't exist
os.makedirs(output_location, exist_ok=True)
video_name = os.path.basename(video_location)
# Open the video and get its properties
cap = cv2.VideoCapture(video_location)
frame_rate = (cap.get(cv2.CAP_PROP_FPS))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(frame_rate)
print(frame_count)

# Initialize a list to store the frames
frame_arr = []
file_names = []


29.97002997002997
25518


In [25]:
# Loop through the frames and save them to the output location

for i in tqdm(range(0, frame_count,round(frame_rate * 3))):
    # Read the frame and convert it to an RGB image
    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Save the frame to the output location with the timestamp as the name
    timestamp_sec = i / frame_rate
    timestamp_min = int(timestamp_sec // 60)
    timestamp_sec = int(timestamp_sec % 60)
    filename = f"{timestamp_min:02d}:{timestamp_sec:02d}.jpg"
    # Add the frame to the list
    frame_arr.append(frame)
    file_names.append(filename)
    cv2.imwrite(os.path.join(output_location, filename), frame)

    
    list_of_images=frame_arr


  0%|          | 0/284 [00:00<?, ?it/s]

In [26]:
len(frame_arr)

284

In [27]:
##Read frames from storage for usage.

import os

def load_images_from_folder(folder):
    images = []
    
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
            file_names.append(filename)
            
    return images

list_of_images = load_images_from_folder("/content/drive/MyDrive/clippy/output/{title}".format(title=video_title))

In [28]:
len(list_of_images)

284

In [29]:
#conveting frames to embeddings


from tqdm.auto import tqdm

batch_size = 16
image_arr = None  

for i in tqdm( range(0, len(list_of_images), batch_size) ) :
    # select batch of images
    batch = list_of_images[ i: i+batch_size]
    # process and resize
    batch = processor(
        text=None,
        images=batch,
        return_tensors='pt',
        padding=True
    )['pixel_values'].to(device )

    # get image embeddings
    batch_emb = model.get_image_features(pixel_values=batch)
    # convert to numpy array
    #batch_emb = batch_emb.squeeze(0)
    batch_emb = batch_emb.cpu().detach().numpy()
    # add to larger array of all image embeddings
    if image_arr is None:
        image_arr = batch_emb
    else:
        image_arr = np.concatenate( (image_arr, batch_emb), axis=0)
image_arr.shape

  0%|          | 0/18 [00:00<?, ?it/s]

(284, 512)

In [30]:
# Conbine Embeddings with Meta Data

type(image_arr)
vectors = []
for i in range(len(image_arr)):
    vectors.append({ 
        "id": f"{video_name}_{file_names[i]}",
        "metadata": { 
            "video_name": video_name,
            "time_stamp": file_names[i]
        },
        "values": image_arr[i].tolist()
    })


In [31]:
len(vectors)

284

In [32]:
#insert (Frame Embeddings + Meta Data) to Pinecone

index.upsert(vectors,namespace=namespace_name)

{'upserted_count': 284}

# Text Query

In [ ]:


prompt = input('Enter Item to be serached in video')

#tokenize the prompt
inputs = tokenizer(prompt, return_tensors="pt")
inputs.to(device)
text_emb = model.get_text_features(**inputs)
text_emb.shape
text_emb

In [34]:
text_emb = text_emb.cpu().detach().numpy()

# Fetch 10 results

In [35]:
query_results = index.query(
  vector=text_emb.tolist(),
  top_k=10,
  include_values=False,
  include_metadata=True,
   namespace=namespace_name,
)

# Printing Results

## Top 10 matches


In [36]:
for match in query_results['matches']:
    print(f"Video: {match['metadata']['video_name']}, Time: {match['metadata']['time_stamp']}, Score: {match['score']}")

Video: Top 10 Most Intelligent Animals.mp4, Time: 10:48.jpg, Score: 34.2771683
Video: Top 20 Most Intelligent Animals.mp4, Time: 06:39.jpg, Score: 33.4636765
Video: Top 10 Most Intelligent Animals.mp4, Time: 05:00.jpg, Score: 33.4142952
Video: Top 20 Most Intelligent Animals.mp4, Time: 06:42.jpg, Score: 32.2302
Video: Top 20 Most Intelligent Animals.mp4, Time: 06:45.jpg, Score: 32.0587616
Video: Top 10 Most Intelligent Animals.mp4, Time: 05:18.jpg, Score: 31.8288879
Video: Top 10 Most Intelligent Animals.mp4, Time: 03:39.jpg, Score: 31.5251045
Video: Top 20 Most Intelligent Animals.mp4, Time: 06:24.jpg, Score: 31.4189129
Video: Top 10 Most Intelligent Animals.mp4, Time: 10:45.jpg, Score: 31.3574047
Video: Top 20 Most Intelligent Animals.mp4, Time: 06:48.jpg, Score: 31.3030338


## Print Video Names & Timestamp - Detailed


In [37]:
unique_video_names = {}
unique_results = []

for match in query_results['matches']:
    video_name = match['metadata']['video_name']
    if video_name not in unique_video_names:
        unique_video_names[video_name] = True
        unique_results.append(match)

print(unique_results)

[{'id': 'Top 10 Most Intelligent Animals.mp4_10:48.jpg',
 'metadata': {'time_stamp': '10:48.jpg',
              'video_name': 'Top 10 Most Intelligent Animals.mp4'},
 'score': 34.2771683,
 'values': []}, {'id': 'Top 20 Most Intelligent Animals.mp4_06:39.jpg',
 'metadata': {'time_stamp': '06:39.jpg',
              'video_name': 'Top 20 Most Intelligent Animals.mp4'},
 'score': 33.4636765,
 'values': []}]


## Print Video Names & Timestamp only

In [38]:
#only name & timestamp
for videoName in unique_results:
  print(videoName['id'])

Top 10 Most Intelligent Animals.mp4_10:48.jpg
Top 20 Most Intelligent Animals.mp4_06:39.jpg


## Print Upto 3 frames from each video

In [39]:
#Upto 3 frames from each video
unique_video_names = {}
unique_results = []

# Keep track of the number of frames added for each video
num_frames_per_video = {}

for match in query_results['matches']:
    video_name = match['metadata']['video_name']
    
    # Add up to 3 unique frames from each video
    if video_name not in unique_video_names:
        unique_video_names[video_name] = True
        unique_results.append(match)
        num_frames_per_video[video_name] = 1
    elif num_frames_per_video[video_name] < 3:
        unique_results.append(match)
        num_frames_per_video[video_name] += 1

print(unique_results)

[{'id': 'Top 10 Most Intelligent Animals.mp4_10:48.jpg',
 'metadata': {'time_stamp': '10:48.jpg',
              'video_name': 'Top 10 Most Intelligent Animals.mp4'},
 'score': 34.2771683,
 'values': []}, {'id': 'Top 20 Most Intelligent Animals.mp4_06:39.jpg',
 'metadata': {'time_stamp': '06:39.jpg',
              'video_name': 'Top 20 Most Intelligent Animals.mp4'},
 'score': 33.4636765,
 'values': []}, {'id': 'Top 10 Most Intelligent Animals.mp4_05:00.jpg',
 'metadata': {'time_stamp': '05:00.jpg',
              'video_name': 'Top 10 Most Intelligent Animals.mp4'},
 'score': 33.4142952,
 'values': []}, {'id': 'Top 20 Most Intelligent Animals.mp4_06:42.jpg',
 'metadata': {'time_stamp': '06:42.jpg',
              'video_name': 'Top 20 Most Intelligent Animals.mp4'},
 'score': 32.2302,
 'values': []}, {'id': 'Top 20 Most Intelligent Animals.mp4_06:45.jpg',
 'metadata': {'time_stamp': '06:45.jpg',
              'video_name': 'Top 20 Most Intelligent Animals.mp4'},
 'score': 32.0587616,
 'v

# Delete a Namespace

In [40]:
namespace_name_to_delete = input(' Enter the Namespace you want to delete')
index.delete(deleteAll='true', namespace=namespace_name_to_delete)

 Enter the Namespace you want to deleteanimals


{}